In [1]:
pip install transformers datasets torch pandas

Note: you may need to restart the kernel to use updated packages.


In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split

# let us load our dataset from CSV
df=pd.read_csv("/kaggle/input/patientemail/patients.csv",encoding="windows-1252")


In [81]:
# let us split the data 
train_df,eval_df=train_test_split(df,test_size=0.2,random_state=42)

In [82]:
train_df.head()

,patient_id,patient_name,problems,medicines_suggested,address,doctor_visited,last_checkup_date,genre_of_doctor,hospitals_visited,blood_group,weight,height,age,email
9254,9255,Elijah Moore,Celiac disease,Sumatriptan,80924 Anthony Island,Dr. Mia Green,2023-08-30,Gastroenterologist,Hearing & Balance Center,O-,62,168,18,"\nRespected Dr. Mia Green,\n\nI hope you're ha..."
1561,1562,Mia Price,Cataracts,Gatifloxacin,33973 Elizabeth Parkway,Dr. Amelia Moore,2024-02-20,Endocrinologist,Evergreen Dermatology,A+,86,188,49,"\nRespected Dr. Amelia Moore,\n\nMy nam..."
1670,1671,Evelyn Taylor,"Thyroid issue, Crohn's disease",Betahistine,30508 Miller Highway,Dr. Henry Robinson,2023-09-13,Orthopedic Surgeon,Skin Disease Center,AB-,93,180,60,"\nRespected Dr. Henry Robinson,\n\nI hope you'..."
6087,6088,Alexander Quinn,"Trigeminal neuralgia, ADHD",Meloxicam,21000 Michelle Boulevard,Dr. Noah Baker,2023-08-09,Dermatologist,Mountain Thyroid Center,A-,86,192,28,"\nRespected Dr. Noah Baker,\n\nMy name ..."
6669,6670,Evelyn Jackson,"Pharyngitis, Psoriasis",Vitamin A Palmitate,18954 William Stream,Dr. Abigail Wright,2023-12-17,General Physician,Skin & Beauty Center,A-,60,168,59,"\nRespected Dr. Abigail Wright,\n\nMy n..."


In [83]:
eval_df.head()

,patient_id,patient_name,problems,medicines_suggested,address,doctor_visited,last_checkup_date,genre_of_doctor,hospitals_visited,blood_group,weight,height,age,email
6252,6253,Abigail Ramirez,Fracture,Phenytoin,46301 Joseph Circle,Dr. Grace Moore,2023-07-12,Gastroenterologist,Dermatology Care Center,A-,71,179,46,"\nRespected Dr. Grace Moore,\n\nI hope you're ..."
4684,4685,Daniel Reed,"Multiple sclerosis, Adrenal insufficiency",Calcipotriol,94963 Joseph Street,Dr. Jackson Robinson,2023-08-27,Ophthalmologist,Voice Disorders Clinic,O-,77,190,40,"\nRespected Dr. Jackson Robinson,\n\nI hope yo..."
1731,1732,Olivia Taylor,"Tendonitis, Crohn's disease",Selenium sulfide,88829 Jacob Place,Dr. Abigail Clark,2023-09-11,Orthopedic Surgeon,Allergy & Skin Care,A+,64,192,67,"\nRespected Dr. Abigail Clark,\n\nMy na..."
4742,4743,Daniel White,"Fever, Multiple sclerosis",Paroxetine,26488 Charles Square,Dr. Sophia Scott,2023-08-06,Ophthalmologist,Bone & Joint Specialists,O-,78,182,64,"\nRespected Dr. Sophia Scott,\n\nI hope you're..."
4521,4522,Logan Evans,"Cushing's syndrome, Coronary artery disease",Lisinopril,97183 Jason Stream,Dr. Mia Wilson,2024-02-19,Cardiologist,Valley Diabetes Clinic,B+,81,166,31,"\nRespected Dr. Mia Wilson,\n\nMy name ..."


In [11]:
# lets see the first few rows of our dataset
print(df.head())

   patient_id    patient_name                         problems  \
0           1     Milo Foster  Astigmatism, Retinal detachment   
1           2     Hazel Quinn   Vertigo, Seborrheic dermatitis   
2           3     Sophia Long         Psoriasis, Thyroid issue   
3           4    Sophia White                        Gastritis   
4           5  Ethan Thompson                        Allergies   

  medicines_suggested                     address      doctor_visited  \
0           Ofloxacin           56323 Ryan Square   Dr. Abigail Young   
1    Physical therapy          41858 Taylor Court    Dr. James Garcia   
2           Emollient        13825 Elizabeth Road     Dr. Amelia King   
3        Valacyclovir  29592 Christopher Crescent   Dr. Michael Davis   
4      Hydrocortisone          15231 Daniel Route  Dr. Michael Garcia   

  last_checkup_date     genre_of_doctor          hospitals_visited  \
0        2024-01-18  Gastroenterologist             Clarity Vision   
1        2024-01-26     

In [85]:
print(len(train_df))
print(len(eval_df))

8000
2000


In [118]:
import random

def generate_prompt(row):
    # List of possible fields to shuffle
    fields = [
        f"Name: {row['patient_name']}",
        f"Age: {row['age']}",
        f"Symptoms: {row['problems']}",
        f"Medicines Suggested: {row['medicines_suggested']}",
        f"Doctor Visited: {row['doctor_visited']}",
        f"Address: {row['address']}",
        f"Last Checkup Date: {row['last_checkup_date']}"
    ]

    # Randomly choose a template type
    template_type = random.choice([1, 2])

    if template_type == 1: # either shuffle stuff
        # Template 1: Shuffled prompt format
        random.shuffle(fields) # the place where shuffling is happening
        prompt = (
            f"Patient Information:\n"
            + "\n".join(fields) + "\n\n"
            f"Please generate a professional email summarizing the above information."
        )

        return prompt
    elif template_type==2: # or different way of seeing stuffs
        templates = [
        (
            f"Patient Details:\n"
            f"Name: {row['patient_name']}\n"
            f"Age: {row['age']}\n"
            f"Symptoms: {row['problems']}\n"
            f"Medicines: {row['medicines_suggested']}\n"
            f"Doctor: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last Visit: {row['last_checkup_date']}\n\n"
            f"Compose a professional email with this information."
        ),
        (
            f"Here's the patient's information:\n"
            f"- Name: {row['patient_name']}\n"
            f"- Age: {row['age']}\n"
            f"- Issues: {row['problems']}\n"
            f"- Prescribed Medicine: {row['medicines_suggested']}\n"
            f"- Doctor Consulted: {row['doctor_visited']}\n"
            f"- Address: {row['address']}\n"
            f"- Last Visit Date: {row['last_checkup_date']}\n\n"
            f"Generate a formal email to the doctor based on this data."
        ),
        (
            f"Using the following patient data, create an email for the doctor:\n"
            f"Name: {row['patient_name']}\n"
            f"Age: {row['age']}\n"
            f"Symptoms: {row['problems']}\n"
            f"Medication: {row['medicines_suggested']}\n"
            f"Consulted with: {row['doctor_visited']}\n"
            f"Patient's Address: {row['address']}\n"
            f"Checkup Date: {row['last_checkup_date']}."
        ),
        (
            f"Patient Name: {row['patient_name']} (Age {row['age']})\n"
            f"Symptoms reported: {row['problems']}\n"
            f"Medicine suggested: {row['medicines_suggested']}\n"
            f"Doctor consulted: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last checkup on: {row['last_checkup_date']}.\n\n"
            f"Write a concise and professional email with this information."
        ),
        (
            f"Name: {row['patient_name']}\n"
            f"Age: {row['age']}\n"
            f"Symptoms: {row['problems']}\n"
            f"Medication: {row['medicines_suggested']}\n"
            f"Address: {row['address']}\n"
            f"Visited: {row['doctor_visited']}\n"
            f"Last Visit: {row['last_checkup_date']}\n\n"
            f"Draft a formal email summarizing these details."
        ),
        (
            f"The patient, {row['patient_name']} (aged {row['age']}), reported the following symptoms: {row['problems']}.\n"
            f"Medicine prescribed: {row['medicines_suggested']}.\n"
            f"Doctor consulted: {row['doctor_visited']}.\n"
            f"Address: {row['address']}.\n"
            f"Last checkup date: {row['last_checkup_date']}.\n\n"
            f"Kindly create a professional email conveying this information."
        ),
        # Additional templates
        (
            f"Patient Information:\n"
            f"Name: {row['patient_name']}\n"
            f"Age: {row['age']}\n"
            f"Symptoms: {row['problems']}\n"
            f"Medicines Suggested: {row['medicines_suggested']}\n"
            f"Doctor: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last Checkup Date: {row['last_checkup_date']}\n\n"
            f"Please generate a professional email summarizing the above information."
        ),
        (
            f"Patient Record:\n"
            f"- Name: {row['patient_name']}\n"
            f"- Age: {row['age']}\n"
            f"- Symptoms: {row['problems']}\n"
            f"- Medicines Suggested: {row['medicines_suggested']}\n"
            f"- Doctor Visited: {row['doctor_visited']}\n"
            f"- Address: {row['address']}\n"
            f"- Last Checkup: {row['last_checkup_date']}\n\n"
            f"Create an email summarizing the patient's information."
        ),
        (
            f"Based on the following patient's information, please write an email:\n"
            f"Name: {row['patient_name']}\n"
            f"Age: {row['age']}\n"
            f"Symptoms: {row['problems']}\n"
            f"Medicines Suggested: {row['medicines_suggested']}\n"
            f"Doctor Visited: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last Checkup Date: {row['last_checkup_date']}.\n\n"
            f"Compose a polite and professional email for the doctor."
        ),
        (
            f"Patient: {row['patient_name']}, Age: {row['age']}\n"
            f"Symptoms: {row['problems']}\n"
            f"Medicines: {row['medicines_suggested']}\n"
            f"Doctor: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last Checkup Date: {row['last_checkup_date']}\n\n"
            f"Generate a formal email summarizing the patient's case."
        ),
        (
            f"Here's the information of a patient:\n"
            f"Name: {row['patient_name']}, Age: {row['age']}, Symptoms: {row['problems']}\n"
            f"Medicines Suggested: {row['medicines_suggested']}\n"
            f"Doctor: {row['doctor_visited']}, Address: {row['address']}\n"
            f"Last Visit: {row['last_checkup_date']}\n\n"
            f"Write a formal email to the doctor summarizing the patient's details."
        ),
        (
            f"Name: {row['patient_name']} (Age: {row['age']})\n"
            f"Symptoms: {row['problems']}\n"
            f"Medicines Prescribed: {row['medicines_suggested']}\n"
            f"Doctor Consulted: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last Checkup: {row['last_checkup_date']}\n\n"
            f"Please draft a professional email with the details above."
        ),
        (
            f"The patient, {row['patient_name']}, aged {row['age']}, is experiencing the following symptoms: {row['problems']}.\n"
            f"Medicines suggested: {row['medicines_suggested']}.\n"
            f"Visited: {row['doctor_visited']}\n"
            f"Address: {row['address']}\n"
            f"Last Checkup: {row['last_checkup_date']}\n\n"
            f"Generate an email summarizing the information provided."
        ),
        (
            f"Patient: {row['patient_name']}, Age: {row['age']}, Symptoms: {row['problems']}\n"
            f"Medicines: {row['medicines_suggested']}, Doctor Visited: {row['doctor_visited']}\n"
            f"Address: {row['address']}, Last Checkup: {row['last_checkup_date']}\n\n"
            f"Write a detailed, professional email for the doctor based on this data."
        ),
        (
            f"My name is {row['patient_name']},my age is {row['age']}, my symptomps are: {row['problems']}\n" #isd
            f"Medicines I am currently taking are: {row['medicines_suggested']}, Doctor I visited last was : {row['doctor_visited']}\n"
            f"I am from {row['address']}, last I had a checkup on: {row['last_checkup_date']}\n"
            f"Write an email to the doctor regrding this."
        ),
        (
           f"My name is {row['patient_name']}, my age is {row['age']}\n"
           f"I am experiencing the following symptoms: {row['problems']}. "
           f"The medicines prescribed to me are: {row['medicines_suggested']}. "
           f"My last checkup was on: {row['last_checkup_date']}. "
           f"I am from {row['address']}. The doctor I visited was Dr. {row['doctor_visited']}. "
           f"Please write an email to the doctor regarding this."

        ),
        (
            f"I am {row['age']} years old, and my name is {row['patient_name']}. "
            f"I have been experiencing the following symptoms: {row['problems']}. "
            f"The medicines I am taking are: {row['medicines_suggested']}. "
            f"I visited Dr. {row['doctor_visited']}, and I live at {row['address']}. "
            f"My last checkup was on {row['last_checkup_date']}. "
            f"Please draft an email to the doctor regarding this."
        ),
        (
        f"My name is {row['patient_name']} and I am {row['age']} years old. "
        f"I am suffering from {row['problems']}. "
        f"The doctor I last visited was Dr. {row['doctor_visited']}. "
        f"The medicines suggested are: {row['medicines_suggested']}. "
        f"I am from {row['address']}, and my last checkup was on {row['last_checkup_date']}. "
        f"Please write an email to the doctor regarding my condition."
        ),
        (
        f"I am {row['age']} years old, my name is {row['patient_name']}. "
        f"My symptoms include: {row['problems']}, and I am taking {row['medicines_suggested']}. "
        f"My last checkup was on {row['last_checkup_date']}, and I live at {row['address']}. "
        f"I last visited Dr. {row['doctor_visited']}. "
        f"Please write an email to the doctor about this."
        ),
        (
        f"I am {row['patient_name']} and I am {row['age']} years old. "
        f"My symptoms are: {row['problems']}. "
        f"The medicines I am currently taking are: {row['medicines_suggested']}. "
        f"I am from {row['address']}, and I saw Dr. {row['doctor_visited']} during my last checkup. "
        f"That was on {row['last_checkup_date']}. "
        f"Please compose an email to the doctor based on this."
        ),
        (
            f"I am {row['age']} years old, and my name is {row['patient_name']}. "
        f"My symptoms are: {row['problems']}. "
        f"The last doctor I visited was Dr. {row['doctor_visited']}. "
        f"I live in {row['address']}, and I am currently taking {row['medicines_suggested']}. "
        f"My last checkup was on {row['last_checkup_date']}. "
        f"Please send an email to the doctor regarding this."
        ),
        (
        f"I am {row['patient_name']}, and I am {row['age']} years old. "
        f"My symptoms are: {row['problems']}. "
        f"I have been prescribed {row['medicines_suggested']}. "
        f"My last checkup was on {row['last_checkup_date']}. "
        f"I live in {row['address']}, and I saw Dr. {row['doctor_visited']} for my last visit. "
        f"Please write an email to the doctor about this.",
        ),
        (
            f"His name is {row['patient_name']}, and he is {row['age']} years old. "
            f"His symptoms include: {row['problems']}. The medicines he is currently taking are: {row['medicines_suggested']}. "
            f"He last visited Dr. {row['doctor_visited']}, and his last checkup was on {row['last_checkup_date']}. "
            f"He resides in {row['address']}. Please write an email to the doctor regarding this."
        ),
        (
            f"Her name is {row['patient_name']}, and she is {row['age']} years old. "
            f"She has been experiencing the following symptoms: {row['problems']}. "
            f"The medicines prescribed to her are: {row['medicines_suggested']}. "
            f"She last visited Dr. {row['doctor_visited']} on {row['last_checkup_date']}. "
            f"She is from {row['address']}. Please draft an email to the doctor based on this information.",

        ),
        (
             f"Last she visited Dr. {row['doctor_visited']}, her symptoms were {row['problems']}. "
             f"Her name is {row['patient_name']}, and she is {row['age']} years old. "
             f"She resides in {row['address']} and was prescribed the following medicines: {row['medicines_suggested']}. "
             f"Her last checkup was on {row['last_checkup_date']}. Please write an email to the doctor about her condition.",

        ),
        (
             f"His name is {row['patient_name']}, aged {row['age']}, and he lives in {row['address']}. "
             f"He is currently dealing with the following symptoms: {row['problems']}. "
             f"The medicines suggested for him are: {row['medicines_suggested']}. "
             f"Last he visited Dr. {row['doctor_visited']} on {row['last_checkup_date']}. "
             f"Please compose an email to the doctor regarding his medical history."
        ),
        (
            f"Her name is {row['patient_name']}, and she is {row['age']} years old. "
            f"She last visited Dr. {row['doctor_visited']}, where she reported these symptoms: {row['problems']}. "
            f"She lives in {row['address']} and is currently taking {row['medicines_suggested']}. "
            f"Her last checkup was conducted on {row['last_checkup_date']}. "
            f"Please draft an email to the doctor with this information."
        )
    ]

    return random.choice(templates)

    

# Example usage:
# row = {'patient_name': 'John Doe', 'age': 30, 'problems': 'Headache', 'medicines_suggested': 'Paracetamol', 
#        'doctor_visited': 'Dr. Smith', 'address': '123 Main St', 'last_checkup_date': '2023-12-15'}

# print(generate_prompt(row))


In [119]:
# applying the function created to make the input text column, the prompt
train_df["input_text"]=df.apply(generate_prompt,axis=1)

# let us create a column for the target text too
train_df["target_text"]=df["email"]

eval_df["input_text"]=df.apply(generate_prompt,axis=1)
eval_df["target_text"]=eval_df["email"]

In [121]:
print(train_df[["input_text","target_text"]][:10])

                                             input_text  \
9254  Patient Information:\nAge: 18\nMedicines Sugge...   
1561  Patient Information:\nDoctor Visited: Dr. Amel...   
1670  My name is Evelyn Taylor, my age is 60\nI am e...   
6087  Here's the information of a patient:\nName: Al...   
6669  Based on the following patient's information, ...   
5933  Patient: Harper Turner, Age: 63, Symptoms: Fev...   
8829  Patient Information:\nSymptoms: Social anxiety...   
7945  Patient Details:\nName: Benjamin Thompson\nAge...   
3508  My name is Evelyn Turner,my age is 11, my symp...   
2002  Patient Information:\nName: Milo Oliver\nAddre...   

                                            target_text  
9254  \nRespected Dr. Mia Green,\n\nI hope you're ha...  
1561         \nRespected Dr. Amelia Moore,\n\nMy nam...  
1670  \nRespected Dr. Henry Robinson,\n\nI hope you'...  
6087         \nRespected Dr. Noah Baker,\n\nMy name ...  
6669         \nRespected Dr. Abigail Wright,\n\nMy n...  
59

In [126]:
print(eval_df[["input_text","target_text"]])

                                             input_text  \
6252  Patient Information:\nName: Abigail Ramirez\nS...   
4684  (Her name is Daniel Reed, and she is 40 years ...   
1731  Patient Information:\nAge: 67\nName: Olivia Ta...   
4742  Name: Daniel White\nAge: 64\nSymptoms: Fever, ...   
4521  My name is Logan Evans, my age is 31\nI am exp...   
...                                                 ...   
6412  Here's the patient's information:\n- Name: Noa...   
8285  Patient Information:\nAge: 62\nAddress: 76422 ...   
7853  Patient Information:\nSymptoms: Otitis externa...   
1095  Patient Information:\nDoctor Visited: Dr. Amel...   
6929  Patient Information:\nLast Checkup Date: 2023-...   

                                            target_text  
6252  \nRespected Dr. Grace Moore,\n\nI hope you're ...  
4684  \nRespected Dr. Jackson Robinson,\n\nI hope yo...  
1731         \nRespected Dr. Abigail Clark,\n\nMy na...  
4742  \nRespected Dr. Sophia Scott,\n\nI hope you're...  
4

In [128]:
print(train_df.dtypes)
print(eval_df.dtypes)


patient_id              int64
patient_name           object
problems               object
medicines_suggested    object
address                object
doctor_visited         object
last_checkup_date      object
genre_of_doctor        object
hospitals_visited      object
blood_group            object
weight                  int64
height                  int64
age                     int64
email                  object
input_text             object
target_text            object
dtype: object
patient_id              int64
patient_name           object
problems               object
medicines_suggested    object
address                object
doctor_visited         object
last_checkup_date      object
genre_of_doctor        object
hospitals_visited      object
blood_group            object
weight                  int64
height                  int64
age                     int64
email                  object
input_text             object
target_text            object
dtype: object


In [129]:
train_df["input_text"] = train_df["input_text"].astype(str)
train_df["target_text"] = train_df["target_text"].astype(str)

eval_df["input_text"] = eval_df["input_text"].astype(str)
eval_df["target_text"] = eval_df["target_text"].astype(str)


In [130]:
from transformers import T5Tokenizer

# loding the pretrained t5 Tokenizer
tokenizer=T5Tokenizer.from_pretrained("t5-base")

# let us tokenize the fucking dataset now
def tokenize_function(examples):
    model_inputs=tokenizer(examples["input_text"],truncation=True,padding="max_length",max_length=512)
    labels=tokenizer(examples["target_text"],truncation=True,padding="max_length",max_length=512)
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs

# let us convert the dataframe now into hugging face dataset


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [131]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train_df[["input_text","target_text"]])
eval_dataset=Dataset.from_pandas(eval_df[["input_text","target_text"]])
# let us apply the tokenization

tokenized_train_dataset=train_dataset.map(tokenize_function,batched=True)
tokenized_eval_dataset=eval_dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [132]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_text', 'target_text', '__index_level_0__'],
    num_rows: 8000
})
Dataset({
    features: ['input_text', 'target_text', '__index_level_0__'],
    num_rows: 2000
})


In [133]:
print(train_dataset[:2])

{'input_text': ['Patient Information:\nAge: 18\nMedicines Suggested: Sumatriptan\nAddress: 80924 Anthony Island\nDoctor Visited: Dr. Mia Green\nLast Checkup Date: 2023-08-30\nSymptoms: Celiac disease\nName: Elijah Moore\n\nPlease generate a professional email summarizing the above information.', 'Patient Information:\nDoctor Visited: Dr. Amelia Moore\nMedicines Suggested: Gatifloxacin\nAddress: 33973 Elizabeth Parkway\nName: Mia Price\nAge: 49\nLast Checkup Date: 2024-02-20\nSymptoms: Cataracts\n\nPlease generate a professional email summarizing the above information.'], 'target_text': ["\nRespected Dr. Mia Green,\n\nI hope you're having a good day. My name is Elijah Moore and I’m writing to provide you with my medical information for your reference. \n\nPatient ID: 9255.\nPatient Name: Elijah Moore.\nAge:18.\nProblems: Celiac disease.\nMedicines Suggested: Sumatriptan.\nAddress: 80924 Anthony Island.\nDoctor Visited: Dr. Mia Green.\nLast Checkup Date: 2023-08-30.\nGenre of Doctor: Gas

In [94]:
# let us load the pretrained model


from transformers import T5ForConditionalGeneration

model=T5ForConditionalGeneration.from_pretrained("t5-small")

In [98]:
from transformers import Trainer,TrainingArguments

training_args=TrainingArguments(
    output_dir="t5emailv2S",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    warmup_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [100]:
print(len(tokenized_train_dataset))

8000


In [101]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

In [102]:
from tqdm import tqdm
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,0.133800,0.093527
1000,0.088400,0.072480
1500,0.080000,0.067379
2000,0.073800,0.064582
2500,0.073100,0.062992
3000,0.072600,0.062491


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3000, training_loss=0.48759065141280494, metrics={'train_runtime': 1739.7598, 'train_samples_per_second': 13.795, 'train_steps_per_second': 1.724, 'total_flos': 3248203235328000.0, 'train_loss': 0.48759065141280494, 'epoch': 3.0})

In [60]:
!pip install wandb

In [65]:
import wandb
#import wandb
wandb.login(key="5e722de16ab75deba10e56428ff403f861370ed4")


wandb: Currently logged in as: treysarkar (treysarkar-auk). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [67]:
wandb.init(project="t5email", name="treysarkar")


In [103]:
eval_results=trainer.evaluate()
print(eval_results)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.06249063089489937, 'eval_runtime': 50.1419, 'eval_samples_per_second': 39.887, 'eval_steps_per_second': 4.986, 'epoch': 3.0}


In [134]:
print(len(tokenized_train_dataset))

8000


In [104]:
trainer.push_to_hub("Training complete")

events.out.tfevents.1736874314.0e62a1761f38.40.5:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/treysarkar/t5emailv2S/commit/abb13de876188eb8b55fff8dad73545a3bfc20d9', commit_message='Training complete', commit_description='', oid='abb13de876188eb8b55fff8dad73545a3bfc20d9', pr_url=None, pr_revision=None, pr_num=None)

In [135]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Load the T5-Base model and tokenizer
model_name = "t5-base"  # T5 Base model, which has more parameters than T5 Small
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Your training arguments
training_args = TrainingArguments(
    output_dir="t5email_base_v2",  # Updated output directory for T5-Base
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    warmup_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    push_to_hub=True,
)

# Assuming you have your tokenized train_dataset
# Use the Trainer class to fine-tune the model with your training data

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Your tokenized training dataset
    eval_dataset=tokenized_eval_dataset,    # Your tokenized evaluation dataset
)

# Start the training process
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,0.082300,0.068536
1000,0.066800,0.058989
1500,0.061400,0.055498
2000,0.059600,0.054434
2500,0.058500,0.053833
3000,0.058100,0.053614


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3000, training_loss=0.43463012701272963, metrics={'train_runtime': 5437.4337, 'train_samples_per_second': 4.414, 'train_steps_per_second': 0.552, 'total_flos': 1.461498937344e+16, 'train_loss': 0.43463012701272963, 'epoch': 3.0})

In [136]:
#not yet executed
eval_res=trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [137]:
print(eval_res)

{'eval_loss': 0.053613997995853424, 'eval_runtime': 150.4857, 'eval_samples_per_second': 13.29, 'eval_steps_per_second': 1.661, 'epoch': 3.0}


In [138]:
trainer.push_to_hub("Training Complete!!")

events.out.tfevents.1736889360.0e62a1761f38.40.10:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/treysarkar/t5email_base_v2/commit/f387a5de2151744e9c0bc2dd37291b29e00f4435', commit_message='Training Complete!!', commit_description='', oid='f387a5de2151744e9c0bc2dd37291b29e00f4435', pr_url=None, pr_revision=None, pr_num=None)

In [139]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the fine-tuned model and tokenizer from Hugging Face Hub
model_name = "treysarkar/t5email_base_v2"  # Your model name
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the function for generating email outputs
def generate_email(prompt, model, tokenizer, max_length=512):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate the output
    outputs = model.generate(
        inputs.input_ids, 
        max_length=max_length, 
        num_beams=5,  # Use beam search for better results
        early_stopping=True
    )
    
    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# List of example prompts
prompts = [
    " Doctor Visited: Dr. Radhika Verma, Age: 34,Patient: Aditi Sharma, Symptoms: Fever, Cough, Headache, Medicines Suggested: Paracetamol, Address: 89 Green Lane, Last Checkup Date: 2023-11-12. Generate a professional email based on this information.",
    # "Patient: Arun Singh, Age: 25, Symptoms: Nausea, Vomiting, Medicines Suggested: Ondansetron, Doctor Visited: Dr. Neha Agarwal, Address: 45 Red Street, Last Checkup Date: 2023-10-30. Generate a professional email based on this information.",
    # "Patient Name: Priya Mehta, Age: 28, Symptoms: Fatigue, Joint Pain, Medicines Suggested: Ibuprofen, Doctor Visited: Dr. Sudhir Kumar, Address: 21 Whitefield Avenue, Last Checkup Date: 2023-09-20. Generate a professional email based on this information.",
    # "Patient: Rajesh Kumar, Age: 41, Symptoms: Chest Pain, Difficulty Breathing, Medicines Suggested: Aspirin, Doctor Visited: Dr. Shalini Gupta, Address: 78 Park Road, Last Checkup Date: 2023-11-05. Generate a professional email based on this information.",
    # "Patient Name: Sunita Yadav, Age: 30, Symptoms: Back Pain, Dizziness, Medicines Suggested: Tylenol, Doctor Visited: Dr. Vikram Singh, Address: 67 Blue Ridge, Last Checkup Date: 2023-12-01. Generate a professional email based on this information."
    # "Patient Name: Sunita Yadav, Age: 30, Symptoms: Back Pain, Dizziness, Medicines Suggested: Tylenol, Doctor Visited: Dr. Vikram Singh, Address: 67 Blue Ridge, Last Checkup Date: 2023-12-01. Generate a professional email based on this information."
    "My name is Rajesh Mukho, my age is 34, symptoms are skin rashes, medicines are duphalac, doctor visited was Dr. Raghuram De, addres: 12 Nilgunj, last checkup on 23-03-23. Write an email to the doctor"
]

# Generate and print emails for each prompt
for i, prompt in enumerate(prompts):
    email = generate_email(prompt, model, tokenizer)
    print(f"Prompt {i+1}:\n{prompt}\nGenerated Email:\n{email}\n{'-'*50}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Prompt 1:
 Doctor Visited: Dr. Radhika Verma, Age: 34,Patient: Aditi Sharma, Symptoms: Fever, Cough, Headache, Medicines Suggested: Paracetamol, Address: 89 Green Lane, Last Checkup Date: 2023-11-12. Generate a professional email based on this information.
Generated Email:
Respected Dr. Radhika Verma, I hope you're having a good day. My name is Aditi Sharma and I’m writing to provide you with my medical information for your reference. Patient ID: 6767. Patient Name: Aditi Sharma. Age:34. Problems: Fever, Cough, Headache. Medicines Suggested: Paracetamol. Address: 89 Green Lane. Doctor Visited: Dr. Radhika Verma. Last Checkup Date: 2023-11-12. Genre of Doctor: Psychiatrist. Hospitals Visited: Skin Wellness Clinic. Blood Group: O+. Weight: 67. Height: 168. Please let me know if you require any additional information or if there are any further steps I should take. Best regards, Aditi Sharma, via MediTalk.
--------------------------------------------------
Prompt 2:
My name is Rajesh Mukh